In [1]:
import os,sys
sys.path.append('../src/')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from simpledbf import Dbf5

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
datapath_in = '../data/raw/logradouros/'
datapath_out = '../data/interim/'

In [3]:
# Loading all dbf files into one single dataframe:
df_list = []

for obj in os.listdir(datapath_in):
    logradouros_path = os.path.join(datapath_in, obj)
    if os.path.isdir(logradouros_path):
        uf = obj.split('_')[0]
        print(f"Loading files for {logradouros_path} folder ({uf})...")
        for filename in os.listdir(logradouros_path):
            if filename[-4:] == '.dbf':                
                filepath = os.path.join(logradouros_path, filename)
                # print(filepath)

                dbf = Dbf5(filepath)
                df = dbf.to_dataframe()
                df['uf'] = uf
                df_list.append(df)
print("Done!")
df_all = pd.concat(df_list)

Loading files for ../data/raw/logradouros/ac_faces_de_logradouros_2021 folder (ac)...
Loading files for ../data/raw/logradouros/al_faces_de_logradouros_2021 folder (al)...
Loading files for ../data/raw/logradouros/am_faces_de_logradouros_2021 folder (am)...
Loading files for ../data/raw/logradouros/ap_faces_de_logradouros_2021 folder (ap)...
Loading files for ../data/raw/logradouros/ba_faces_de_logradouros_2021 folder (ba)...
Loading files for ../data/raw/logradouros/ce_faces_de_logradouros_2021 folder (ce)...
Loading files for ../data/raw/logradouros/df_faces_de_logradouros_2021 folder (df)...
Loading files for ../data/raw/logradouros/es_faces_de_logradouros_2021 folder (es)...
Loading files for ../data/raw/logradouros/go_faces_de_logradouros_2021 folder (go)...
Loading files for ../data/raw/logradouros/ma_faces_de_logradouros_2021 folder (ma)...
Loading files for ../data/raw/logradouros/mg_faces_de_logradouros_2021 folder (mg)...
Loading files for ../data/raw/logradouros/ms_faces_de_

In [4]:
df_all

,CD_SETOR,CD_QUADRA,CD_FACE,NM_TIP_LOG,NM_TIT_LOG,NM_LOG,TOT_RES,TOT_GERAL,uf
0,120001305000001,001,001,AVENIDA,GOVERNADOR,EDMUNDO PINTO,10.0,11.0,ac
1,120001305000001,001,002,RUA,NaN,RIO DE JANEIRO,5.0,5.0,ac
2,120001305000001,001,003,RUA,NaN,MINAS GERAIS,10.0,10.0,ac
3,120001305000001,001,004,AVENIDA,NaN,ADENILSOM ROGERIO DE OLIVEIRA,2.0,2.0,ac
4,120001305000001,002,001,RUA,NaN,RIO DE JANEIRO,4.0,5.0,ac
...,...,...,...,...,...,...,...,...,...
963,172210705000029,030,002,RUA,NaN,SEM DENOMINACAO,1.0,1.0,to
964,172210705000029,030,003,AVENIDA,NaN,A,1.0,1.0,to
965,172210705000029,030,004,RUA,NaN,CINCO,1.0,2.0,to
966,172210705000029,031,002,AVENIDA,NaN,A,0.0,0.0,to


In [5]:
# df = pd.DataFrame({'name': ['Raphael', 'Donatello'],
#                    'mask': ['red', 'purple'],
#                    'weapon': ['sai', 'bo staff']})
# df.to_csv(index=False)


In [6]:
# df = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4]})
# df.to_parquet('df.parquet.gzip',
#               compression='gzip')  
# pd.read_parquet('df.parquet.gzip')  

In [7]:
# Saving file to output folder:
df_all.to_csv(os.path.join(datapath_out, 'logradouros.csv'), index=False)
df_all.to_parquet(os.path.join(datapath_out, 'logradouros.parquet.gzip'), compression='gzip')